In [1]:
import sys
sys.path.insert(1, '../pyKinectAzure/')
from pygame import mixer

import numpy as np
from pyKinectAzure import pyKinectAzure, _k4a
import matplotlib.pyplot as plt
import cv2
from scipy import ndimage as ndi
from scipy.ndimage.measurements import label
import itertools
import time 
from playsound import playsound 

modulePath = 'C:\\Program Files\\Azure Kinect SDK v1.4.1\\sdk\\windows-desktop\\amd64\\release\\bin\\k4a.dll' 

pyK4A = pyKinectAzure(modulePath)


pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
m = 1

In [3]:
pyK4A.device_open()

device_config = pyK4A.config

# device_config.color_resolution = _k4a.K4A_COLOR_RESOLUTION_1536P
device_config.color_resolution = _k4a.K4A_COLOR_RESOLUTION_720P
device_config.depth_mode = _k4a.K4A_DEPTH_MODE_NFOV_UNBINNED
device_config.camera_fps = _k4a.K4A_FRAMES_PER_SECOND_30
print("Okay")

pyK4A.device_start_cameras(device_config)

while True:
    pyK4A.device_get_capture()

    color_image_handle = pyK4A.capture_get_color_image()

    if color_image_handle:

        color_image = pyK4A.image_convert_to_numpy(color_image_handle)
  
        cv2.imshow('Colorized Depth Image',color_image)

        pyK4A.image_release(color_image_handle)

    pyK4A.capture_release()

    if cv2.waitKey(20) & 0xFF==ord('d'):
        cv2.destroyAllWindows()
        break

pyK4A.device_stop_cameras()
pyK4A.device_close()

cv2.imwrite('D:\dataset_1\mask\mask_{}_720.png'.format(m), color_image)
# маску которая идет к 47-53 я просрала, к 1м 4м тоже нет
m += 1


Okay


In [4]:
pyK4A.device_open()

device_config = pyK4A.config

# device_config.color_resolution = _k4a.K4A_COLOR_RESOLUTION_1536P
device_config.color_resolution = _k4a.K4A_COLOR_RESOLUTION_720P
device_config.depth_mode = _k4a.K4A_DEPTH_MODE_NFOV_UNBINNED
device_config.camera_fps = _k4a.K4A_FRAMES_PER_SECOND_30
print("Okay")

pyK4A.device_start_cameras(device_config)

i = 149
while True:
    pyK4A.device_get_capture()

    depth_image_handle = pyK4A.capture_get_depth_image()
    color_image_handle = pyK4A.capture_get_color_image()


    if color_image_handle and depth_image_handle:

        color_image = pyK4A.image_convert_to_numpy(color_image_handle)
        transformed_depth_image = pyK4A.transform_depth_to_color(depth_image_handle,color_image_handle)
        transformed_depth_color_image = cv2.applyColorMap(np.round(transformed_depth_image/5).astype(np.uint8), cv2.COLORMAP_JET)

        cv2.imshow('Colorized Depth Image',color_image)

        pyK4A.image_release(depth_image_handle)
        pyK4A.image_release(color_image_handle)
        
        mixer.init()
        mixer.music.load('screenshot.mp3')
        mixer.music.play()
        
        cv2.imwrite('D:\dataset_1\color\img_{}_720.png'.format(i), color_image)
        cv2.imwrite('D:\dataset_1\depth_rgb\depth_color_{}_720.png'.format(i), transformed_depth_color_image)
    
        # save depth image as txt.
        np.savetxt('D:\dataset_1\depth_txt\depth_txt_{}_720.txt'.format(i), transformed_depth_image)
    
        # save rgb imsge as txt. Use it only for 3d arrays cause 1 d we can save easier
        with open('D:\dataset_1\color_txt\color_{}_720.txt'.format(i), 'w') as outfile:
            for slice_2d in color_image:
                np.savetxt(outfile, slice_2d)

    pyK4A.capture_release()
    
    i += 1
    print(i)
    
#     time.sleep(2) 
    

    if cv2.waitKey(20) & 0xFF==ord('d'):
        cv2.destroyAllWindows()
        break

pyK4A.device_stop_cameras()
pyK4A.device_close()

Okay
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398